# 言語モデルとRNN

## 言語モデル

embeddingを直接推定するのではなく、単語予測モデル（言語モデル）を構築して副次的にembeddingを取得することになる。

言語モデルとは、尤もらしい文章を生成できるような確率分布を習得するために、文章の確率を推定するモデルのこと。

ある文章$S$をトークン化したのを$(w_1, w_2, \cdots, w_n)$と表記するならば、

$$
P(S)=P(w_1, w_2, \cdots, w_n)
$$

を求めたいということになる。これは条件付き確率の積として表せる

$$
\begin{align}
P(w_1, w_2, \cdots, w_n)
&= P(w_1) \times P(w_2|w_1) \times P(w_3|w_1, w_2) \times \cdots\\
&= \prod_{i=1}^n p(w_i|\boldsymbol{c}_i)
\end{align}
$$

ここで$\boldsymbol{c}_i$は$w_i$より前のトークン列$\boldsymbol{c}_i=(w_1,w_2,\cdots,w_{i-1})$で、**文脈**（context）と呼ばれる



:::{note} 同時確率の分解

これは確率の乗法定理

$$
P(A,B) = P(A|B) P(B)
$$

に基づく。


$w_m$までの単語を$C_m$とすると、

$$
P(\underbrace{w_1, \dots, w_{m-1}}_{C_m}, w_m)
= P(C_m, w_m)
= P(w_m | C_m) P(C_m)
$$

さらに

$$
P(C_m) = 
P(\underbrace{w_1, \dots, w_{m-2}}_{C_{m-1}}, w_{m-1})
= P(C_{m-1}, w_{m-1})
= P(w_{m-1} | C_{m-1}) P(C_{m-1})
$$

となる。これを繰り返すことで上記のようになる
:::